## Mean / Median imputation

[Feature Engineering for Machine Learning Course](https://www.trainindata.com/p/feature-engineering-for-machine-learning)

Imputation is the act of replacing missing data with statistical estimates of the missing values. The goal of any imputation technique is to produce a **complete dataset** that can be used to train machine learning models.

Mean/median imputation consists of replacing all occurrences of missing values (NA) within a variable by the mean (if the variable has a Gaussian distribution) or median (if the variable has a skewed distribution).

**Note**:

If a variable is normally distributed, the mean, median, and mode are approximately the same. Therefore, replacing missing values by the mean and the median are equivalent.

If the variable is skewed, the mean is biased towards values at the end of the distribution. As a result, the median represents the majority of the values in the variable better.

Replacing missing data by the mode is not common practice for numerical variables.

### Which variables can I impute with mean or median imputation?

The mean and median can only be calculated on numerical variables. Therefore, these methods are suitable for continuous and discrete numerical variables only.

### Assumptions

- Data is missing completely at random.

If data is missing at random, it's safe to assume that the missing values are extremely close to the mean or median of the distribution, which is the value of the majority of the observations.

### Advantages

- Fast way of obtaining complete datasets

### Limitations

- Distortion of the original variable distribution (if the fraction of NA is large):

- Distortion of the variance

- Distortion of the covariance with other variables of the dataset

If the proportion of NA is large, the variance of the variable will be distorted when it is replaced with the mean or median, resulting in an underestimating of the variance.

Estimates of covariance and correlations with other variables may also be affected. Because the mean or median value that now substitutes the missing data does not always retain the relation with the remaining variables, mean or median imputation may change intrinsic correlations.

Finally, "concentrating" missing values at the mean or median value may lead to observations that are common occurrences in the distribution being picked up as outliers.

### When should mean or median imputation be used?

- Data is missing completely at random

- No more than 5% of the variable contains missing data

Although in theory, the above conditions should be met to minimise the impact of this imputation technique, in practice, mean / median imputation is very commonly used, even when data is not MCAR and the fraction of missing values is large. The reason behind this, is the simplicity of the technique.

### Final note

Replacement of NA with mean or median is widely used in the data science community and in data science competitions. See, for example, the winning solution of the KDD 2009 cup: ["Winning the KDD Cup Orange Challenge with Ensemble Selection"](http://www.mtome.com/Publications/CiML/CiML-v3-book.pdf).

Typically, mean and median imputation is done together with adding a "missing indicator" variable to mark observations where the data was missing (see lecture "Missing Indicator"), thus covering 2 angles: if the data was missing completely at random, this would be captured by the mean and median imputation, and if it wasn't, this would be captured by the additional "missing indicator" variable. Both methods are extremely straightforward to implement and therefore are a top choice for imputation.

## Datasets:

- Ames House Price
- Titanic

To download the datasets, please refer to the lecture **Datasets** in **Section 2** of this course.


In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

# to split the datasets
from sklearn.model_selection import train_test_split

## Titanic


In [3]:
# Load the Titanic dataset with a few variables for demonstration.

data = pd.read_csv("../../Datasets/titanic.csv", usecols=["age", "fare", "survived"])
data.head()

,survived,age,fare
0,1,29.0000,211.3375
1,1,0.9167,151.5500
2,0,2.0000,151.5500
3,0,30.0000,151.5500
4,0,25.0000,151.5500


In [4]:
# Let's look at the fraction of NA.

data.isnull().mean()

survived    0.000000
age         0.200917
fare        0.000764
dtype: float64

Age and fare show missing data. If instead of `mean()`, we use `sum()`, we will see that fare only has 1 missing value.

### Imputation important

The imputation values (that is the mean/median) should be calculated using the training set, and the same value should be used to impute the test set. This is to avoid overfitting.


In [ ]:
# Let's separate into training and testing set.

X_train, X_test, y_train, y_test = train_test_split(
    data[["age", "fare"]],  # predictors
    data["survived"],  # target
    test_size=0.5,  # percentage of obs in test set
    random_state=0,
)  # seed to ensure reproducibility

X_train.shape, X_test.shape

In [ ]:
# Let's explore the missing data in the train set.
# The percentage of NA should be fairly similar to those
# observed in the whole dataset.

X_train.isnull().mean()

In [ ]:
# Calculate the median Age.

median = X_train.age.median()
median

In [ ]:
# Calculate the mean Age

mean = X_train.age.mean()

# The mean contains many decimals, so I round to 1
# using the round function from numpy.
mean = np.round(mean, 1)

mean

In [ ]:
# Create a new variable with the missing values replaced
# using the function we created above.

# First, replace with the median.
X_train["Age_median"] = X_train["age"].fillna(median)

# Now replace with the mean.
X_train["Age_mean"] = X_train["age"].fillna(mean)

X_train.head(15)

Look at the rows with missing data (NaN) in Age, and see how in the new variables those were replaced by either 28 (median) or 30 (mean).


In [ ]:
# We can see a change in the variance after mean / median imputation.
# This is expected, because the percentage of missing data is quite
# high in Age, ~20%.

print("Original variable variance: ", X_train["age"].var())
print("Variance after median imputation: ", X_train["Age_median"].var())
print("Variance after mean imputation: ", X_train["Age_mean"].var())

As expected, the variance is underestimated because many values are now the same (i.e., the mean or the median value).


In [ ]:
# We can see that the distribution has changed:
# there are now more values towards the median
# or median.

fig = plt.figure()
ax = fig.add_subplot(111)

# original variable distribution
X_train["age"].plot(kind="kde", ax=ax)

# variable imputed with the median
X_train["Age_median"].plot(kind="kde", ax=ax, color="red")

# variable imputed with the mean
X_train["Age_mean"].plot(kind="kde", ax=ax, color="green")

# add legends
lines, labels = ax.get_legend_handles_labels()
ax.legend(lines, labels, loc="best")

As mentioned previously, the mean / median imputation distorted the original distribution of the variable Age. The transformed variable shows more values around the mean / median values.

**Is this important?**

It depends on the machine learning model you want to build. Distorting the variable distribution may affect its relationship with the target, which might be important for linear models. Therefore, the final imputed variable might negatively affect the linear model's performance.


In [ ]:
# We also said that mean / median imputation may affect the relationship
# with the other variables in the dataset.

# Let's have a look.

X_train[["fare", "age", "Age_median", "Age_mean"]].cov()

The covariance between Age and Fare changed with the mean / median imputation.


In [ ]:
# Finally, I mentioned that mean / median imputation may lead
# inliers to look like outliers.

# In other words, mean / median imputation may lead to an increase
# in the number of detected outliers.

# Let's find out using a boxplot.
X_train[["age", "Age_median", "Age_mean"]].boxplot()

From the boxplot, we can see that after the imputation, we not only have more outliers at the higher age values, but we now have outliers as well at the lower age values.

**Is this important?**

If we are after true outliers, we need to keep this behavior in mind to make sure that we are neither masking nor creating artificial outliers with our imputation technique. In practice, we normally don't check for this behaviour at all. But I think it is important to know that it is happening.

## House Prices Dataset

In this dataset, there are variables with a lot or with a few missing data points. Therefore, we can compare the effects of mean and median imputation in both cases.


In [ ]:
# We'll use the following variables,
# 3 of which contain NA.

cols_to_use = [
    "OverallQual",
    "TotalBsmtSF",
    "1stFlrSF",
    "GrLivArea",
    "WoodDeckSF",
    "BsmtUnfSF",
    "LotFrontage",
    "MasVnrArea",
    "GarageYrBlt",
    "SalePrice",
]

In [ ]:
# Let's load the House Prices dataset.

data = pd.read_csv("../../Datasets/houseprice.csv", usecols=cols_to_use)
print(data.shape)
data.head()

In [ ]:
# print the variables with missing data.

# In the next list comprehension, I loop over all the columns
# of the dataset and select those with missing data.

[var for var in data.columns if data[var].isnull().sum() > 0]

In [ ]:
# Find the percentage of missing data in those variables
# (same code used in section 3).

data[["LotFrontage", "MasVnrArea", "GarageYrBlt"]].isnull().mean()

**Remember that the mean and the median that we will use to replace the NA are calculated using the train set.**


In [ ]:
# Let's separate into training and testing sets.

X_train, X_test, y_train, y_test = train_test_split(
    data.drop("SalePrice", axis=1),
    data["SalePrice"],
    test_size=0.3,
    random_state=0,
)

X_train.shape, X_test.shape

In [ ]:
# Let's plot the distributions of the variables.
# We learned this code in section 3.

X_train.hist(bins=50, figsize=(10, 10))
plt.show()

In [ ]:
# Create a new variable with missing values replaced
# by the median using the function we created previously.

# I repeat the following lines for each of the variables with NA
# over the next cells.

median = X_train["LotFrontage"].median()

X_train["LotFrontage_median"] = X_train["LotFrontage"].fillna(median)
X_test["LotFrontage_median"] = X_test["LotFrontage"].fillna(median)

In [ ]:
median = X_train["MasVnrArea"].median()

X_train["MasVnrArea_median"] = X_train["MasVnrArea"].fillna(median)
X_test["MasVnrArea_median"] = X_test["MasVnrArea"].fillna(median)

In [ ]:
median = X_train["GarageYrBlt"].median()

X_train["GarageYrBlt_median"] = X_train["GarageYrBlt"].fillna(median)
X_test["GarageYrBlt_median"] = X_test["GarageYrBlt"].fillna(median)

In [ ]:
# Let's evaluate the effect of median imputation on the distribution.

# We can see that the distribution has changed for LotFrontAge,
# there are more values towards the median.

fig = plt.figure()
ax = fig.add_subplot(111)

# Original variable distribution.
X_train["LotFrontage"].plot(kind="kde", ax=ax)

# Variable imputed with the median.
X_train["LotFrontage_median"].plot(kind="kde", ax=ax, color="red")

# Add legends.
lines, labels = ax.get_legend_handles_labels()
ax.legend(lines, labels, loc="best")

In [ ]:
# For MasVnrArea, because the percentage of missing data
# was low, we don't expect and won't see a dramatic change in the
# distribution of the variable.

fig = plt.figure()
ax = fig.add_subplot(111)

# original variable distribution
X_train["MasVnrArea"].plot(kind="kde", ax=ax)

# variable imputed with the median
X_train["MasVnrArea_median"].plot(kind="kde", ax=ax, color="red")

# add legends
lines, labels = ax.get_legend_handles_labels()
ax.legend(lines, labels, loc="best")

In [ ]:
# Similarly, for GarageBlt, because the percentage of missing data
# was low, we don't expect a big change in the
# distribution. However, we see more values
# around the median.

fig = plt.figure()
ax = fig.add_subplot(111)

# original variable distribution
X_train["GarageYrBlt"].plot(kind="kde", ax=ax)

# variable imputed with the median
X_train["GarageYrBlt_median"].plot(kind="kde", ax=ax, color="red")

# add legends
lines, labels = ax.get_legend_handles_labels()
ax.legend(lines, labels, loc="best")

In [ ]:
# We can also explore the effect of the imputation technique on
# the variance.

# We can see a change in the variance after imputation of LotFrontAge.
# This is expected, because the percentage of missing data is quite
# high ~20%.

print("Original Variance: ", X_train["LotFrontage"].var())
print("Variance after median imputation: ", X_train["LotFrontage_median"].var())

In [ ]:
# This is not the case for MasnVnrArea,
# because the % of missing data was small.

print("Original Variance: ", X_train["MasVnrArea"].var())
print("Variance after median imputation: ", X_train["MasVnrArea_median"].var())

In [ ]:
# For GarageBlt, the % of missing data was small, so the effect
# on the variance will also be small.

print("Original Variance: ", X_train["GarageYrBlt"].var())
print("Variance after median imputation: ", X_train["GarageYrBlt_median"].var())

In [ ]:
# Finally, let's explore the covariance:
# Take your time to compare the values in the table below.
# See how the covariance is affected for LotFrontAge but not
# so much for the other 2 variables.

X_train.cov()

In [ ]:
# Finally, let's explore the effect on outliers.

# There is a decrease in the outlier boundaries for LotFrontage
# (look at the height of the top whiskers),
# but there is not a visible effect for the other 2 variables.

X_train[["LotFrontage", "LotFrontage_median"]].boxplot()

In [ ]:
X_train[["MasVnrArea", "MasVnrArea_median"]].boxplot()

In [ ]:
X_train[["GarageYrBlt", "GarageYrBlt_median"]].boxplot()

Later in this section, we will carry out mean / median imputation with open source Python packages.
